In [ ]:
import os
import time
import csv
from pathlib import Path

import requests
import pandas as pd
from tqdm import tqdm

# =========================================================
# QA CONFIG
# =========================================================
STRICT = False  # True = raise errors; False = log warnings and continue

KEY_COLS = [
    "id", "lpa_name", "valid_date", "decision", "status",
    "application_type", "postcode"
]

# =========================================================
# API CONFIG
# =========================================================
API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

# =========================================================
# BOROUGHS (canonical for lpa_name.raw)
# =========================================================
BOROUGHS = [
    "Barking & Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "City of London", "Croydon", "Ealing", "Enfield",
    "Greenwich", "Hackney", "Hammersmith & Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington & Chelsea", "Kingston", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth",
    "Westminster"
]

START_YEAR, END_YEAR = 2010, 2025

# =========================================================
# OUTPUT
# =========================================================
OUT_DIR = r"c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/planning_extract_core_parts"
os.makedirs(OUT_DIR, exist_ok=True)
QA_LOG = os.path.join(OUT_DIR, "_qa_log.csv")

# =========================================================
# FIELDS (CORE EXTRACT)
# =========================================================
FIELDS = [
    "id", "lpa_app_no", "case_reference", "planning_portal_reference",
    "lpa_name", "ward", "postcode", "site_name", "site_number", "uprn",
    "received_date", "valid_date", "decision_date", "lapsed_date", "last_updated",
    "application_type", "application_type_full", "development_type",
    "decision", "status", "decision_process",
    "centroid_easting", "centroid_northing", "centroid_latitude", "centroid_longitude",
    "latitude", "longitude"
]

# =========================================================
# QA HELPERS
# =========================================================
def _to_dt(series: pd.Series) -> pd.Series:
    """Parse mixed date formats for QA only. Does not affect saved file."""
    try:
        return pd.to_datetime(series, errors="coerce", utc=True, dayfirst=True, format="mixed")
    except TypeError:
        return pd.to_datetime(series, errors="coerce", utc=True, dayfirst=True)

def qa_chunk(df: pd.DataFrame, borough: str, year: int) -> dict:
    outfile = f"planning_core_{borough.replace('&','and').replace(' ','_')}_{year}.csv"
    report = {"borough": borough, "year": int(year), "outfile": outfile}

    n = len(df)
    report["rows"] = int(n)

    # IDs
    if "id" in df.columns:
        id_col = df["id"]
        report["id_missing_pct"] = float(id_col.isna().mean())
        report["id_distinct"] = int(id_col.nunique(dropna=True))
        report["id_dup_rows"] = int(id_col.dropna().duplicated().sum())
    else:
        report["id_missing_pct"] = 1.0
        report["id_distinct"] = 0
        report["id_dup_rows"] = 0

    # Missingness for core columns
    for c in KEY_COLS:
        report[f"{c}_missing_pct"] = float(df[c].isna().mean()) if c in df.columns else 1.0

    # Date sanity + year check (QA only)
    if "valid_date" in df.columns and n > 0:
        vd = _to_dt(df["valid_date"])
        report["valid_date_min"] = str(vd.min()) if vd.notna().any() else ""
        report["valid_date_max"] = str(vd.max()) if vd.notna().any() else ""
        report["valid_date_parse_fail_pct"] = float(vd.isna().mean())

        parsed = vd.dropna()
        if len(parsed) > 0:
            report["valid_date_outside_year_pct"] = float((parsed.dt.year != year).mean())
        else:
            report["valid_date_outside_year_pct"] = 0.0
    else:
        report["valid_date_min"] = ""
        report["valid_date_max"] = ""
        report["valid_date_parse_fail_pct"] = 1.0
        report["valid_date_outside_year_pct"] = 0.0

    # Flags
    flags = []

    if n == 0:
        flags.append("EMPTY_CHUNK")

    # tiny chunks are suspicious (tune threshold if you want)
    if 0 < n < 50:
        flags.append("LOW_VOLUME_LT50")

    if report["id_missing_pct"] > 0.001:
        flags.append("MISSING_IDS")

    if report["id_dup_rows"] > 0:
        flags.append("DUPLICATE_IDS")

    # valid_date should exist (you filter on it)
    if report.get("valid_date_missing_pct", 1.0) > 0.01:
        flags.append("VALID_DATE_MISSING_HIGH")

    if report["valid_date_parse_fail_pct"] > 0.05:
        flags.append("VALID_DATE_PARSE_FAIL_HIGH")

    if report["valid_date_outside_year_pct"] > 0.01:
        flags.append("VALID_DATE_OUTSIDE_YEAR")

    for must in ["lpa_name", "status", "decision"]:
        if must not in df.columns:
            flags.append(f"MISSING_COL_{must}")

    report["flags"] = "|".join(flags)
    report["ok"] = (len(flags) == 0)
    return report

def qa_log_row(row: dict) -> None:
    """Append QA report to QA_LOG CSV, writing header once."""
    file_exists = Path(QA_LOG).exists()

    # Keep column order stable (prevents messy CSV headers)
    fieldnames = list(row.keys())

    with open(QA_LOG, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            w.writeheader()
        w.writerow(row)

def handle_qa(report: dict) -> None:
    """Print warnings or raise if STRICT."""
    if report.get("ok", True):
        return

    msg = (
        f"[QA] {report.get('borough')} {report.get('year')} "
        f"FLAGS={report.get('flags')} rows={report.get('rows')} "
        f"file={report.get('outfile')}"
    )

    if STRICT:
        raise RuntimeError(msg)
    else:
        print(msg)

# =========================================================
# EXTRACTION HELPERS
# =========================================================
def year_iter(start: int, end: int):
    for y in range(start, end + 1):
        yield f"01/01/{y}", f"01/01/{y+1}", y

def fetch_borough_year(session: requests.Session, borough: str, gte_date: str, lt_date: str, size: int = 1000):
    rows, page = [], 0
    while True:
        query = {
            "query": {"bool": {"must": [
                {"term": {"lpa_name.raw": borough}},
                {"range": {"valid_date": {"gte": gte_date, "lt": lt_date}}}
            ]}},
            "_source": FIELDS,
            "from": page * size,
            "size": size
        }

        r = session.post(API, headers=HDR, json=query, timeout=60)
        r.raise_for_status()

        hits = r.json().get("hits", {}).get("hits", [])
        if not hits:
            break

        rows.extend(h.get("_source", {}) for h in hits)

        if len(hits) < size:
            break

        page += 1
        time.sleep(0.05)  # polite pacing

    return rows

# =========================================================
# MAIN
# =========================================================
def main():
    with requests.Session() as session:
        for borough in BOROUGHS:
            print(f"\n=== {borough} ===")
            for gte, lt, year in tqdm(list(year_iter(START_YEAR, END_YEAR)), desc="Years"):
                data = fetch_borough_year(session, borough, gte, lt)
                print(f"{year}: {len(data)} records")

                if not data:
                    # Still log QA row for visibility (optional; comment out if you don't want it)
                    df_empty = pd.DataFrame(columns=FIELDS)
                    rep = qa_chunk(df_empty, borough, year)
                    qa_log_row(rep)
                    handle_qa(rep)
                    continue

                df = pd.DataFrame(data)

                rep = qa_chunk(df, borough, year)
                qa_log_row(rep)
                handle_qa(rep)

                tag = f"{borough.replace('&','and').replace(' ','_')}_{year}"
                out_path = os.path.join(OUT_DIR, f"planning_core_{tag}.csv")
                df.to_csv(out_path, index=False)

    print("\nDONE.")
    print(f"Parts written to: {OUT_DIR}")
    print(f"QA log written to : {QA_LOG}")

if __name__ == "__main__":
    main()



=== Barking & Dagenham ===


Years:   6%|▋         | 1/16 [00:00<00:02,  6.62it/s]

2010: 3 records
[QA] Barking & Dagenham 2010 FLAGS=LOW_VOLUME_LT50 rows=3 file=planning_core_Barking_and_Dagenham_2010.csv
2011: 14 records
[QA] Barking & Dagenham 2011 FLAGS=LOW_VOLUME_LT50 rows=14 file=planning_core_Barking_and_Dagenham_2011.csv
2012: 13 records
[QA] Barking & Dagenham 2012 FLAGS=LOW_VOLUME_LT50 rows=13 file=planning_core_Barking_and_Dagenham_2012.csv


Years:  19%|█▉        | 3/16 [00:00<00:01,  9.10it/s]

2013: 17 records


...

2023: 7960 records


Years:  94%|█████████▍| 15/16 [00:13<00:01,  1.74s/it]

2024: 8904 records


Years: 100%|██████████| 16/16 [00:15<00:00,  1.00it/s]

2025: 8489 records

DONE.
Parts written to: c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/planning_extract_core_parts
QA log written to : c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/planning_extract_core_parts\_qa_log.csv


In [ ]:
import requests
import pandas as pd

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

BOROUGHS = [
    "Barking & Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "City of London", "Croydon", "Ealing", "Enfield",
    "Greenwich", "Hackney", "Hammersmith & Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington & Chelsea", "Kingston", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth",
    "Westminster"
]

# Optional: add aliases when QA shows weird low/zero for pre-2019
BOROUGH_ALIASES = {
    "Barking & Dagenham": ["Barking & Dagenham", "Barking and Dagenham"],
    "Hammersmith & Fulham": ["Hammersmith & Fulham", "Hammersmith and Fulham"],
    "Kensington & Chelsea": ["Kensington & Chelsea", "Kensington and Chelsea"],
    "Kingston": ["Kingston", "Kingston upon Thames"],
    "Richmond": ["Richmond", "Richmond upon Thames"],
}

def borough_terms(borough: str):
    return BOROUGH_ALIASES.get(borough, [borough])

def agg_check_borough(session: requests.Session, borough: str):
    names = borough_terms(borough)

    # overall min valid_date & min decision_date, plus % with valid_date
    q_all = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names}}
        ]}},
        "aggs": {
            "min_valid_date": {"min": {"field": "valid_date"}},
            "min_decision_date": {"min": {"field": "decision_date"}},
            "has_valid_date": {"filter": {"exists": {"field": "valid_date"}}},
        }
    }

    # count pre-2019 by decision_date (fallback lens)
    q_pre2019_decision = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names}},
            {"range": {"decision_date": {"lt": "01/01/2019"}}}
        ]}}
    }

    # count pre-2019 by valid_date (your extraction lens)
    q_pre2019_valid = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names}},
            {"range": {"valid_date": {"lt": "01/01/2019"}}}
        ]}}
    }

    r_all = session.post(API, headers=HDR, json=q_all, timeout=60); r_all.raise_for_status()
    r_dec = session.post(API, headers=HDR, json=q_pre2019_decision, timeout=60); r_dec.raise_for_status()
    r_val = session.post(API, headers=HDR, json=q_pre2019_valid, timeout=60); r_val.raise_for_status()

    j_all = r_all.json()
    total = j_all["hits"]["total"]["value"]
    has_valid = j_all["aggregations"]["has_valid_date"]["doc_count"]
    pct_valid = (has_valid / total) if total else 0.0

    return {
        "borough": borough,
        "names_used": "|".join(names),
        "total_docs": total,
        "pct_has_valid_date": pct_valid,
        "min_valid_date": j_all["aggregations"]["min_valid_date"].get("value_as_string"),
        "min_decision_date": j_all["aggregations"]["min_decision_date"].get("value_as_string"),
        "pre2019_by_valid_date": r_val.json()["hits"]["total"]["value"],
        "pre2019_by_decision_date": r_dec.json()["hits"]["total"]["value"],
    }

def run_checks():
    out = []
    with requests.Session() as session:
        for b in BOROUGHS:
            out.append(agg_check_borough(session, b))
    df = pd.DataFrame(out).sort_values(["pre2019_by_valid_date", "total_docs"], ascending=[True, False])
    return df

df_checks = run_checks()
print(df_checks.shape)
print(df_checks.head(10))


(33, 8)
                 borough                                   names_used  \
19  Kensington & Chelsea  Kensington & Chelsea|Kensington and Chelsea   
28                Sutton                                       Sutton   
12  Hammersmith & Fulham  Hammersmith & Fulham|Hammersmith and Fulham   
20              Kingston                Kingston|Kingston upon Thames   
13              Haringey                                     Haringey   
22              Lewisham                                     Lewisham   
10             Greenwich                                    Greenwich   
24                Newham                                       Newham   
7                Croydon                                      Croydon   
16            Hillingdon                                   Hillingdon   

    total_docs  pct_has_valid_date min_valid_date min_decision_date  \
19       39606            0.908170     02/01/2019        02/01/1987   
28       20275            0.814254     18/12/2

In [3]:
import pandas as pd

# df_checks already exists from your run_checks()

# Parse min_valid_date safely (dayfirst because your strings look dd/mm/yyyy)
df = df_checks.copy()
df["min_valid_date_dt"] = pd.to_datetime(df["min_valid_date"], errors="coerce", dayfirst=True)

# --- Define trainability criteria (edit thresholds if needed)
MIN_PRE2019 = 500  # <-- change to 1000 if you want stricter
HAS_2010 = df["min_valid_date_dt"] <= pd.Timestamp("2019-12-31")
HAS_ENOUGH_PRE2019 = df["pre2019_by_valid_date"].fillna(0).astype(int) >= MIN_PRE2019

df["trainable_2010_2019_by_valid_date"] = HAS_2010 & HAS_ENOUGH_PRE2019

total = len(df)
trainable = int(df["trainable_2010_2019_by_valid_date"].sum())
not_trainable = total - trainable

print("Total boroughs:", total)
print("Trainable 2019-2024 (by valid_date):", trainable)
print("NOT trainable 2019-2024 (by valid_date):", not_trainable)

print("\nNOT trainable list:")
print(df.loc[~df["trainable_2010_2019_by_valid_date"], ["borough","min_valid_date","pre2019_by_valid_date","pct_has_valid_date"]]
      .sort_values(["min_valid_date","pre2019_by_valid_date"], ascending=[True, True]))


Total boroughs: 33
Trainable 2019-2024 (by valid_date): 15
NOT trainable 2019-2024 (by valid_date): 18

NOT trainable list:
                 borough min_valid_date  pre2019_by_valid_date  \
31            Wandsworth     01/12/1999                    231   
19  Kensington & Chelsea     02/01/2019                      0   
23                Merton     02/05/1986                    457   
11               Hackney     03/11/1994                     91   
16            Hillingdon     06/03/2001                     85   
32           Westminster     08/08/2002                    189   
24                Newham     09/05/2005                     15   
9                Enfield     11/02/2004                    213   
15              Havering     14/02/2000                    106   
22              Lewisham     14/09/2010                     15   
13              Haringey     16/03/1983                     11   
14                Harrow     16/04/1992                    194   
7                C

In [8]:
import requests
import pandas as pd
from tqdm import tqdm

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

BOROUGHS = [
    "Barking & Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "City of London", "Croydon", "Ealing", "Enfield",
    "Greenwich", "Hackney", "Hammersmith & Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington & Chelsea", "Kingston", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth",
    "Westminster"
]

BOROUGH_ALIASES = {
    "Barking & Dagenham": ["Barking & Dagenham", "Barking and Dagenham"],
    "Hammersmith & Fulham": ["Hammersmith & Fulham", "Hammersmith and Fulham"],
    "Kensington & Chelsea": ["Kensington & Chelsea", "Kensington and Chelsea"],
    "Kingston": ["Kingston", "Kingston upon Thames"],
    "Richmond": ["Richmond", "Richmond upon Thames"],
}

def names_for(b):
    return BOROUGH_ALIASES.get(b, [b])

def count_year(session, borough, year):
    gte = f"01/01/{year}"
    lt  = f"01/01/{year+1}"
    q = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names_for(borough)}},
            {"range": {"valid_date": {"gte": gte, "lt": lt}}}
        ]}}
    }
    r = session.post(API, headers=HDR, json=q, timeout=60)
    r.raise_for_status()
    return r.json()["hits"]["total"]["value"]

# ---- define "trainable" rules
YEARS = list(range(2010, 2020))
MIN_ROWS_PER_YEAR = 200     # <-- adjust (100/200/500). 200 is a sane starting point.
MIN_YEARS_PRESENT = 8       # need at least 8/10 non-trivial years
MIN_TOTAL_ROWS = 3000       # optional: overall mass in 2010-2019

rows = []
with requests.Session() as session:
    for b in tqdm(BOROUGHS, desc="Boroughs"):
        counts = []
        for y in YEARS:
            n = count_year(session, b, y)
            counts.append(n)
        total = sum(counts)
        years_ok = sum(1 for n in counts if n >= MIN_ROWS_PER_YEAR)

        rows.append({
            "borough": b,
            "total_2010_2019": total,
            "years_ge_min": years_ok,
            "min_year_count": min(counts),
            "max_year_count": max(counts),
            "counts_2010_2019": counts
        })

df_train = pd.DataFrame(rows)
df_train["trainable"] = (
    (df_train["years_ge_min"] >= MIN_YEARS_PRESENT) &
    (df_train["total_2010_2019"] >= MIN_TOTAL_ROWS)
)

print("Total boroughs:", len(df_train))
print("Trainable (year-coverage rule):", int(df_train["trainable"].sum()))
print("NOT trainable:", int((~df_train["trainable"]).sum()))

print("\nNOT trainable list (worst first):")
print(df_train.loc[~df_train["trainable"], ["borough","years_ge_min","total_2010_2019","min_year_count","max_year_count"]]
      .sort_values(["years_ge_min","total_2010_2019"]))


Boroughs: 100%|██████████| 33/33 [00:20<00:00,  1.57it/s]

Total boroughs: 33
Trainable (year-coverage rule): 5
NOT trainable: 28

NOT trainable list (worst first):
                 borough  years_ge_min  total_2010_2019  min_year_count  \
31            Wandsworth             0              226               3   
14                Harrow             0              233               6   
0     Barking & Dagenham             1              850               3   
11               Hackney             1             2193               2   
28                Sutton             1             2444               0   
23                Merton             1             2660              12   
20              Kingston             1             2775               0   
13              Haringey             1             3065               0   
2                 Bexley             1             3121              21   
10             Greenwich             1             3125               0   
15              Havering             1             3252              

In [9]:
# df_train is from your year-coverage run
trainable = df_train[df_train["trainable"]].copy()
print("Trainable boroughs:")
print(trainable[["borough","years_ge_min","total_2010_2019","min_year_count","max_year_count"]]
      .sort_values(["years_ge_min","total_2010_2019"], ascending=False))



Trainable boroughs:
      borough  years_ge_min  total_2010_2019  min_year_count  max_year_count
18  Islington            10            40326            3319            4529
1      Barnet            10            13916             236            7258
8      Ealing             9            44017             199            6345
17   Hounslow             8            17514             131            4173
5      Camden             8             4126             150            1624


In [10]:
def show_counts(borough_name):
    row = df_train[df_train["borough"] == borough_name].iloc[0]
    counts = row["counts_2010_2019"]
    years = list(range(2010, 2020))
    print(borough_name, "years_ge_min=", row["years_ge_min"], "total=", row["total_2010_2019"])
    for y, n in zip(years, counts):
        print(y, n)

# Try a few
show_counts("Barnet")
show_counts("Camden")
show_counts("Westminster")


Barnet years_ge_min= 10 total= 13916
2010 236
2011 283
2012 294
2013 464
2014 601
2015 888
2016 1078
2017 1272
2018 1542
2019 7258
Camden years_ge_min= 8 total= 4126
2010 180
2011 150
2012 201
2013 353
2014 388
2015 387
2016 343
2017 253
2018 247
2019 1624
Westminster years_ge_min= 1 total= 9680
2010 2
2011 4
2012 2
2013 3
2014 10
2015 12
2016 19
2017 37
2018 80
2019 9511


In [11]:
import requests
import pandas as pd
from tqdm import tqdm

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

BOROUGHS = [
    "Barking & Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "City of London", "Croydon", "Ealing", "Enfield",
    "Greenwich", "Hackney", "Hammersmith & Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington & Chelsea", "Kingston", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth",
    "Westminster"
]

BOROUGH_ALIASES = {
    "Barking & Dagenham": ["Barking & Dagenham", "Barking and Dagenham"],
    "Hammersmith & Fulham": ["Hammersmith & Fulham", "Hammersmith and Fulham"],
    "Kensington & Chelsea": ["Kensington & Chelsea", "Kensington and Chelsea"],
    "Kingston": ["Kingston", "Kingston upon Thames"],
    "Richmond": ["Richmond", "Richmond upon Thames"],
}

def names_for(b):
    return BOROUGH_ALIASES.get(b, [b])

def count_year_by_field(session, borough, year, field):
    gte = f"01/01/{year}"
    lt  = f"01/01/{year+1}"
    q = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names_for(borough)}},
            {"range": {field: {"gte": gte, "lt": lt}}}
        ]}}
    }
    r = session.post(API, headers=HDR, json=q, timeout=60)
    r.raise_for_status()
    return r.json()["hits"]["total"]["value"]

YEARS = list(range(2010, 2020))
MIN_ROWS_PER_YEAR = 200
MIN_YEARS_PRESENT = 8
MIN_TOTAL_ROWS = 3000

rows = []
with requests.Session() as session:
    for b in tqdm(BOROUGHS, desc="Boroughs"):
        counts = [count_year_by_field(session, b, y, "decision_date") for y in YEARS]
        total = sum(counts)
        years_ok = sum(1 for n in counts if n >= MIN_ROWS_PER_YEAR)
        rows.append({
            "borough": b,
            "total_2010_2019_decision_date": total,
            "years_ge_min_decision_date": years_ok,
            "min_year_count_decision_date": min(counts),
            "max_year_count_decision_date": max(counts),
            "counts_2010_2019_decision_date": counts
        })

df_dec = pd.DataFrame(rows)
df_dec["trainable_decision_date"] = (
    (df_dec["years_ge_min_decision_date"] >= MIN_YEARS_PRESENT) &
    (df_dec["total_2010_2019_decision_date"] >= MIN_TOTAL_ROWS)
)

print("Trainable (decision_date) 2010-2019:", int(df_dec["trainable_decision_date"].sum()))
print("NOT trainable:", int((~df_dec["trainable_decision_date"]).sum()))

print("\nTrainable boroughs (decision_date):")
print(df_dec[df_dec["trainable_decision_date"]][
    ["borough","years_ge_min_decision_date","total_2010_2019_decision_date",
     "min_year_count_decision_date","max_year_count_decision_date"]
].sort_values(["years_ge_min_decision_date","total_2010_2019_decision_date"], ascending=False))


Boroughs: 100%|██████████| 33/33 [00:21<00:00,  1.54it/s]

Trainable (decision_date) 2010-2019: 11
NOT trainable: 22

Trainable boroughs (decision_date):
        borough  years_ge_min_decision_date  total_2010_2019_decision_date  \
8        Ealing                          10                          44961   
18    Islington                          10                          36631   
17     Hounslow                          10                          17361   
1        Barnet                          10                          15015   
32  Westminster                          10                          11894   
4       Bromley                          10                           8112   
7       Croydon                          10                           8076   
5        Camden                          10                           4752   
11      Hackney                          10                           4680   
31   Wandsworth                          10                           3573   
21      Lambeth                           9    

In [12]:
import requests
import pandas as pd
from tqdm import tqdm

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

TRAINABLE = [
    "Ealing","Islington","Hounslow","Barnet","Westminster","Bromley",
    "Croydon","Camden","Hackney","Wandsworth","Lambeth"
]

BOROUGH_ALIASES = {
    "Barking & Dagenham": ["Barking & Dagenham", "Barking and Dagenham"],
    "Hammersmith & Fulham": ["Hammersmith & Fulham", "Hammersmith and Fulham"],
    "Kensington & Chelsea": ["Kensington & Chelsea", "Kensington and Chelsea"],
    "Kingston": ["Kingston", "Kingston upon Thames"],
    "Richmond": ["Richmond", "Richmond upon Thames"],
}

def names_for(b):
    return BOROUGH_ALIASES.get(b, [b])

def agg_year(session, borough, year):
    gte = f"01/01/{year}"
    lt  = f"01/01/{year+1}"

    # total decisions in that year
    q_total = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names_for(borough)}},
            {"range": {"decision_date": {"gte": gte, "lt": lt}}}
        ]}}
    }

    # suspicious early decisions (before 1990) — should basically be 0
    q_bad = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names_for(borough)}},
            {"range": {"decision_date": {"gte": gte, "lt": lt}}},
            {"range": {"decision_date": {"lt": "01/01/1990"}}}
        ]}}
    }

    rt = session.post(API, headers=HDR, json=q_total, timeout=60); rt.raise_for_status()
    rb = session.post(API, headers=HDR, json=q_bad, timeout=60);   rb.raise_for_status()

    total = rt.json()["hits"]["total"]["value"]
    bad   = rb.json()["hits"]["total"]["value"]
    return total, bad

rows = []
with requests.Session() as session:
    for b in tqdm(TRAINABLE, desc="Trainable boroughs"):
        for y in range(2010, 2020):
            total, bad = agg_year(session, b, y)
            rows.append({"borough": b, "year": y, "decision_count": total, "bad_pre1990": bad})

df_q = pd.DataFrame(rows)
df_q["bad_pre1990_pct"] = df_q["bad_pre1990"] / df_q["decision_count"].replace(0, pd.NA)

print(df_q.groupby("borough")[["decision_count","bad_pre1990"]].sum().sort_values("decision_count", ascending=False))
print("\nWorst borough-years by bad_pre1990_pct:")
print(df_q.sort_values("bad_pre1990_pct", ascending=False).head(20))


Trainable boroughs: 100%|██████████| 11/11 [00:17<00:00,  1.57s/it]

             decision_count  bad_pre1990
borough                                 
Ealing                44961            0
Islington             36631            0
Hounslow              17361            0
Barnet                15015            0
Westminster           11894            0
Bromley                8112            0
Croydon                8076            0
Lambeth                5949            0
Camden                 4752            0
Hackney                4680            0
Wandsworth             3573            0

Worst borough-years by bad_pre1990_pct:
    borough  year  decision_count  bad_pre1990  bad_pre1990_pct
0    Ealing  2010             448            0              0.0
69  Croydon  2019            4202            0              0.0
80  Hackney  2010             327            0              0.0
79   Camden  2019            1175            0              0.0
78   Camden  2018             284            0              0.0
77   Camden  2017             339         

In [13]:
import requests
import pandas as pd
from tqdm import tqdm

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

TRAINABLE = [
    "Ealing","Islington","Hounslow","Barnet","Westminster",
    "Bromley","Croydon","Camden","Hackney","Wandsworth","Lambeth"
]

def count_query(session, borough, year, extra_must=None):
    gte = f"01/01/{year}"
    lt  = f"01/01/{year+1}"
    must = [
        {"term": {"lpa_name.raw": borough}},
        {"range": {"decision_date": {"gte": gte, "lt": lt}}}
    ]
    if extra_must:
        must += extra_must

    q = {"size": 0, "track_total_hits": True, "query": {"bool": {"must": must}}}
    r = session.post(API, headers=HDR, json=q, timeout=60)
    r.raise_for_status()
    return r.json()["hits"]["total"]["value"]

rows = []
with requests.Session() as session:
    for b in tqdm(TRAINABLE, desc="Boroughs"):
        for y in range(2010, 2020):
            total = count_query(session, b, y)
            missing_decision = count_query(session, b, y, extra_must=[{"bool": {"must_not": [{"exists": {"field": "decision"}}]}}])
            missing_status   = count_query(session, b, y, extra_must=[{"bool": {"must_not": [{"exists": {"field": "status"}}]}}])

            rows.append({
                "borough": b, "year": y, "total": total,
                "missing_decision": missing_decision,
                "missing_status": missing_status,
                "missing_decision_pct": (missing_decision / total) if total else None,
                "missing_status_pct": (missing_status / total) if total else None,
            })

df_miss = pd.DataFrame(rows)
print(df_miss.sort_values(["missing_decision_pct"], ascending=False).head(20))
print("\nBy borough (avg pct):")
print(df_miss.groupby("borough")[["missing_decision_pct","missing_status_pct"]].mean().sort_values("missing_decision_pct", ascending=False))


Boroughs: 100%|██████████| 11/11 [00:21<00:00,  1.96s/it]

         borough  year  total  missing_decision  missing_status  \
22      Hounslow  2012    343                26               0   
50       Bromley  2010    307                19               1   
20      Hounslow  2010    257                11               0   
101      Lambeth  2011    194                 7               0   
60       Croydon  2010    362                13               0   
100      Lambeth  2010    251                 8               0   
51       Bromley  2011    274                 7               0   
31        Barnet  2011    474                12               0   
23      Hounslow  2013   1082                27               0   
56       Bromley  2016    470                 9               0   
48   Westminster  2018    326                 6              22   
1         Ealing  2011    347                 6               0   
52       Bromley  2012    291                 5               1   
63       Croydon  2013    380                 6               

In [7]:
import requests
import pandas as pd

# -------------------------
# CONFIG
# -------------------------
API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

BOROUGHS = [
    "Barking & Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "City of London", "Croydon", "Ealing", "Enfield",
    "Greenwich", "Hackney", "Hammersmith & Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington & Chelsea", "Kingston", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth",
    "Westminster"
]

# Optional: aliases when the API has inconsistent lpa_name values
BOROUGH_ALIASES = {
    "Barking & Dagenham": ["Barking & Dagenham", "Barking and Dagenham"],
    "Hammersmith & Fulham": ["Hammersmith & Fulham", "Hammersmith and Fulham"],
    "Kensington & Chelsea": ["Kensington & Chelsea", "Kensington and Chelsea"],
    "Kingston": ["Kingston", "Kingston upon Thames"],
    "Richmond": ["Richmond", "Richmond upon Thames"],
}

TRAIN_START = "01/01/2019"
TRAIN_END_EXCL = "01/01/2025"   # train: [2019-01-01, 2025-01-01)
TEST_START = "01/01/2025"
TEST_END_EXCL = "01/01/2026"    # test:  [2025-01-01, 2026-01-01)

# "Trainable" thresholds (tune if you want)
MIN_TRAIN = 5000   # minimum decisions in 2019-2024
MIN_TEST = 500     # minimum decisions in 2025

# -------------------------
# HELPERS
# -------------------------
def borough_terms(borough: str):
    return BOROUGH_ALIASES.get(borough, [borough])

def agg_check_borough(session: requests.Session, borough: str) -> dict:
    names = borough_terms(borough)

    # overall min valid_date & min decision_date, plus % coverage
    q_all = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names}}
        ]}},
        "aggs": {
            "min_valid_date": {"min": {"field": "valid_date"}},
            "min_decision_date": {"min": {"field": "decision_date"}},
            "has_valid_date": {"filter": {"exists": {"field": "valid_date"}}},
            "has_decision_date": {"filter": {"exists": {"field": "decision_date"}}},
        }
    }

    # TRAIN: 2019-2024 by decision_date
    q_train = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names}},
            {"range": {"decision_date": {"gte": TRAIN_START, "lt": TRAIN_END_EXCL}}}
        ]}}
    }

    # TEST: 2025 by decision_date
    q_test = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"terms": {"lpa_name.raw": names}},
            {"range": {"decision_date": {"gte": TEST_START, "lt": TEST_END_EXCL}}}
        ]}}
    }

    r_all = session.post(API, headers=HDR, json=q_all, timeout=60); r_all.raise_for_status()
    r_train = session.post(API, headers=HDR, json=q_train, timeout=60); r_train.raise_for_status()
    r_test = session.post(API, headers=HDR, json=q_test, timeout=60); r_test.raise_for_status()

    j_all = r_all.json()
    total = j_all["hits"]["total"]["value"]

    has_valid = j_all["aggregations"]["has_valid_date"]["doc_count"]
    has_decision = j_all["aggregations"]["has_decision_date"]["doc_count"]

    pct_valid = (has_valid / total) if total else 0.0
    pct_decision = (has_decision / total) if total else 0.0

    return {
        "borough": borough,
        "names_used": "|".join(names),
        "total_docs": int(total),
        "pct_has_valid_date": float(pct_valid),
        "pct_has_decision_date": float(pct_decision),
        "min_valid_date": j_all["aggregations"]["min_valid_date"].get("value_as_string"),
        "min_decision_date": j_all["aggregations"]["min_decision_date"].get("value_as_string"),
        "train_2019_2024_by_decision_date": int(r_train.json()["hits"]["total"]["value"]),
        "test_2025_by_decision_date": int(r_test.json()["hits"]["total"]["value"]),
    }

def run_checks():
    out = []
    with requests.Session() as session:
        for b in BOROUGHS:
            out.append(agg_check_borough(session, b))

    df = pd.DataFrame(out).sort_values(
        ["train_2019_2024_by_decision_date", "test_2025_by_decision_date", "total_docs"],
        ascending=[True, True, False]
    )
    return df

In [8]:
df_checks = run_checks()
print(df_checks.shape)
print(df_checks.head(10))


(33, 9)
                 borough                                   names_used  \
0     Barking & Dagenham      Barking & Dagenham|Barking and Dagenham   
6         City of London                               City of London   
14                Harrow                                       Harrow   
5                 Camden                                       Camden   
11               Hackney                                      Hackney   
23                Merton                                       Merton   
31            Wandsworth                                   Wandsworth   
29         Tower Hamlets                                Tower Hamlets   
28                Sutton                                       Sutton   
12  Hammersmith & Fulham  Hammersmith & Fulham|Hammersmith and Fulham   

    total_docs  pct_has_valid_date  pct_has_decision_date min_valid_date  \
0        14069            0.673538               0.665719     23/07/1998   
6        10620            0.854896  

In [9]:
df = df_checks.copy()

# parse mins (strings look like dd/mm/yyyy)
df["min_decision_date_dt"] = pd.to_datetime(df["min_decision_date"], errors="coerce", dayfirst=True)

train_counts = df["train_2019_2024_by_decision_date"].fillna(0).astype(int)
test_counts = df["test_2025_by_decision_date"].fillna(0).astype(int)

df["trainable_2019_2024_test_2025"] = (train_counts >= MIN_TRAIN) & (test_counts >= MIN_TEST)

total = len(df)
trainable = int(df["trainable_2019_2024_test_2025"].sum())
not_trainable = total - trainable

print("\nTotal boroughs:", total)
print(f"Trainable (decision_date) train 2019-2024 + test 2025: {trainable}")
print("NOT trainable:", not_trainable)

print("\nNOT trainable list (lowest volumes first):")
print(
    df.loc[~df["trainable_2019_2024_test_2025"],
           ["borough", "min_decision_date", "train_2019_2024_by_decision_date", "test_2025_by_decision_date",
            "pct_has_decision_date"]]
    .sort_values(["train_2019_2024_by_decision_date", "test_2025_by_decision_date"], ascending=[True, True])
)

print("\nTrainable boroughs:")
print(
    df.loc[df["trainable_2019_2024_test_2025"],
           ["borough", "train_2019_2024_by_decision_date", "test_2025_by_decision_date",
            "pct_has_decision_date"]]
    .sort_values(["train_2019_2024_by_decision_date"], ascending=False)
)



Total boroughs: 33
Trainable (decision_date) train 2019-2024 + test 2025: 32
NOT trainable: 1

NOT trainable list (lowest volumes first):
              borough min_decision_date  train_2019_2024_by_decision_date  \
0  Barking & Dagenham        05/01/1948                              4595   

   test_2025_by_decision_date  pct_has_decision_date  
0                         265               0.665719  

Trainable boroughs:
                 borough  train_2019_2024_by_decision_date  \
32           Westminster                             47231   
1                 Barnet                             39817   
4                Bromley                             30688   
8                 Ealing                             30410   
19  Kensington & Chelsea                             30143   
7                Croydon                             27834   
26              Richmond                             27121   
25             Redbridge                             25332   
16            Hil

In [13]:
import requests

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

q = {
    "size": 0,
    "track_total_hits": True,
    "query": {
        "wildcard": {"lpa_name.raw": "*Dagenham*"}
    },
    "aggs": {
        "lpa_names_raw": {
            "terms": {"field": "lpa_name.raw", "size": 50}
        }
    }
}

r = requests.post(API, headers=HDR, json=q, timeout=60)

if r.status_code != 200:
    print("STATUS:", r.status_code)
    print("ERROR BODY:\n", r.text)  # this will tell you exactly what's wrong
    r.raise_for_status()

aggs = r.json()["aggregations"]["lpa_names_raw"]["buckets"]

print("Distinct lpa_name.raw values containing 'Dagenham':")
for b in aggs:
    print(f"- {b['key']}: {b['doc_count']}")


Distinct lpa_name.raw values containing 'Dagenham':
- Barking & Dagenham: 14069


In [16]:
import requests

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

borough = "Barking & Dagenham"

q = {
    "size": 0,
    "track_total_hits": True,
    "query": {
        "bool": {
            "must": [
                {"term": {"lpa_name.raw": borough}},
                {"exists": {"field": "decision_date"}},
                {
                    "range": {
                        "decision_date": {
                            "gte": "01/01/2019",
                            "lt":  "01/01/2026",
                            "format": "dd/MM/yyyy"
                        }
                    }
                }
            ]
        }
    },
    "aggs": {
        "by_year": {
            "date_histogram": {
                "field": "decision_date",
                "calendar_interval": "year",
                "format": "yyyy"
            }
        }
    }
}

r = requests.post(API, headers=HDR, json=q, timeout=60)
if r.status_code != 200:
    print("STATUS:", r.status_code)
    print(r.text)
    r.raise_for_status()

buckets = r.json()["aggregations"]["by_year"]["buckets"]
for b in buckets:
    print(b["key_as_string"], b["doc_count"])


2019 565
2020 920
2021 948
2022 748
2023 773
2024 641
2025 265


In [17]:
import requests

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

borough = "Barking & Dagenham"

q = {
    "size": 0,
    "query": {"bool": {"must": [
        {"term": {"lpa_name.raw": borough}},
        {"exists": {"field": "decision_date"}},
        {"range": {"decision_date": {"gte": "01/01/2025", "lt": "01/01/2026", "format": "dd/MM/yyyy"}}}
    ]}},
    "aggs": {
        "by_month": {
            "date_histogram": {
                "field": "decision_date",
                "calendar_interval": "month",
                "format": "yyyy-MM"
            }
        },
        "max_decision_date": {"max": {"field": "decision_date"}}
    }
}

r = requests.post(API, headers=HDR, json=q, timeout=60)
r.raise_for_status()

aggs = r.json()["aggregations"]
print("max_decision_date:", aggs["max_decision_date"].get("value_as_string"))

for b in aggs["by_month"]["buckets"]:
    print(b["key_as_string"], b["doc_count"])


max_decision_date: 24/09/2025
2025-01 55
2025-02 28
2025-03 57
2025-04 50
2025-05 74
2025-06 0
2025-07 0
2025-08 0
2025-09 1


In [18]:
q2 = {
    "size": 0,
    "query": {"bool": {"must": [
        {"term": {"lpa_name.raw": borough}},
        {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026", "format": "dd/MM/yyyy"}}}
    ]}},
    "aggs": {
        "has_decision_date": {"filter": {"exists": {"field": "decision_date"}}}
    }
}

r2 = requests.post(API, headers=HDR, json=q2, timeout=60)
r2.raise_for_status()
j2 = r2.json()
print("total_2025_by_valid_date:", j2["hits"]["total"]["value"])
print("has_decision_date_2025:", j2["aggregations"]["has_decision_date"]["doc_count"])


total_2025_by_valid_date: 1013
has_decision_date_2025: 188


In [1]:
import requests

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

borough = "Barking & Dagenham"

q = {
    "size": 0,
    "track_total_hits": True,
    "query": {"bool": {"must": [
        {"term": {"lpa_name.raw": borough}},
        {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026", "format": "dd/MM/yyyy"}}}
    ]}},
    "aggs": {
        "has_decision_date": {"filter": {"exists": {"field": "decision_date"}}},
        "missing_decision_date": {
            "filter": {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
        },
        "missing_decision_date_by_status": {
            "filter": {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}},
            "aggs": {"status_terms": {"terms": {"field": "status.raw", "size": 50}}}
        },
        "missing_decision_date_has_decision_field": {
            "filter": {"bool": {"must": [
                {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}},
                {"exists": {"field": "decision"}}
            ]}}
        }
    }
}

r = requests.post(API, headers=HDR, json=q, timeout=60)
r.raise_for_status()
aggs = r.json()["aggregations"]

total = r.json()["hits"]["total"]["value"]
print("TOTAL (valid_date 2025):", total)
print("HAS decision_date:", aggs["has_decision_date"]["doc_count"])
print("MISSING decision_date:", aggs["missing_decision_date"]["doc_count"])
print("MISSING decision_date BUT decision field exists:", aggs["missing_decision_date_has_decision_field"]["doc_count"])

print("\nStatus breakdown where decision_date is missing:")
for b in aggs["missing_decision_date_by_status"]["status_terms"]["buckets"]:
    print(f"- {b['key']}: {b['doc_count']}")


TOTAL (valid_date 2025): 1029
HAS decision_date: 188
MISSING decision_date: 841
MISSING decision_date BUT decision field exists: 0

Status breakdown where decision_date is missing:
- Application Under Consideration: 841


In [2]:
import requests
import pandas as pd
from datetime import datetime

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

BOROUGHS = [
    "Barking & Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "City of London", "Croydon", "Ealing", "Enfield",
    "Greenwich", "Hackney", "Hammersmith & Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington & Chelsea", "Kingston", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth",
    "Westminster"
]

# Only needed if you ever see borough-name mismatches.
# Your later test suggests lpa_name.raw is consistent, but this is harmless.
BOROUGH_ALIASES = {
    "Barking & Dagenham": ["Barking & Dagenham", "Barking and Dagenham"],
    "Hammersmith & Fulham": ["Hammersmith & Fulham", "Hammersmith and Fulham"],
    "Kensington & Chelsea": ["Kensington & Chelsea", "Kensington and Chelsea"],
    "Kingston": ["Kingston", "Kingston upon Thames"],
    "Richmond": ["Richmond", "Richmond upon Thames"],
}

def borough_terms(borough: str):
    return BOROUGH_ALIASES.get(borough, [borough])

def post_json(session: requests.Session, payload: dict, timeout=60):
    r = session.post(API, headers=HDR, json=payload, timeout=timeout)
    if r.status_code >= 400:
        # Print full error text so you can immediately see parse issues (e.g., wrong date format)
        raise requests.HTTPError(f"HTTP {r.status_code}: {r.text}", response=r)
    return r.json()

def borough_2025_outcome_completeness(session: requests.Session, borough: str):
    names = borough_terms(borough)

    # IMPORTANT: this API expects dd/MM/yyyy (you already got burned by yyyy-mm-dd)
    start_2025 = "01/01/2025"
    start_2026 = "01/01/2026"

    # Base filter: applications with valid_date in 2025, for this borough
    base_filter = {
        "bool": {"must": [
            {"terms": {"lpa_name.raw": names}},
            {"range": {"valid_date": {"gte": start_2025, "lt": start_2026}}}
        ]}
    }

    q = {
        "size": 0,
        "track_total_hits": True,
        "query": base_filter,
        "aggs": {
            # total 2025 by valid_date is hits.total
            "has_decision_date": {"filter": {"exists": {"field": "decision_date"}}},

            "missing_decision_date": {
                "filter": {"bool": {"must": [
                    {"terms": {"lpa_name.raw": names}},
                    {"range": {"valid_date": {"gte": start_2025, "lt": start_2026}}},
                    {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
                ]}}
            },

            # what statuses dominate among missing decision_date?
            "status_breakdown_missing_decision_date": {
                "filter": {"bool": {"must": [
                    {"terms": {"lpa_name.raw": names}},
                    {"range": {"valid_date": {"gte": start_2025, "lt": start_2026}}},
                    {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
                ]}},
                "aggs": {
                    "by_status": {"terms": {"field": "status.keyword", "size": 20}}
                }
            },
        }
    }

    j = post_json(session, q)

    total_2025 = j["hits"]["total"]["value"]
    decided_2025 = j["aggregations"]["has_decision_date"]["doc_count"]
    missing_2025 = total_2025 - decided_2025
    pct_decided = (decided_2025 / total_2025) if total_2025 else 0.0

    # pull top status for missing decision_date
    buckets = j["aggregations"]["status_breakdown_missing_decision_date"]["by_status"]["buckets"]
    top_status = buckets[0]["key"] if buckets else ""
    top_status_count = buckets[0]["doc_count"] if buckets else 0
    top_status_pct_missing = (top_status_count / missing_2025) if missing_2025 else 0.0

    return {
        "borough": borough,
        "names_used": "|".join(names),
        "total_valid_date_2025": int(total_2025),
        "has_decision_date_2025": int(decided_2025),
        "missing_decision_date_2025": int(missing_2025),
        "pct_decided_2025": float(pct_decided),
        "top_missing_status": top_status,
        "top_missing_status_count": int(top_status_count),
        "top_missing_status_pct_of_missing": float(top_status_pct_missing),
    }

def run_boroughwide_2025_check(min_pct_decided=0.70, min_decided_abs=200):
    """
    Eligibility rule (edit if needed):
      - pct_decided_2025 >= min_pct_decided
      - AND has_decision_date_2025 >= min_decided_abs (avoid tiny denominators)
    """
    rows = []
    with requests.Session() as session:
        for b in BOROUGHS:
            try:
                rows.append(borough_2025_outcome_completeness(session, b))
            except Exception as e:
                rows.append({
                    "borough": b,
                    "names_used": "|".join(borough_terms(b)),
                    "error": str(e)
                })

    df = pd.DataFrame(rows)

    # If there were errors, keep them visible
    if "error" not in df.columns:
        df["error"] = ""

    # Eligibility flags
    df["eligible_2025_test"] = (
        (df["pct_decided_2025"].fillna(0) >= min_pct_decided) &
        (df["has_decision_date_2025"].fillna(0).astype(float) >= float(min_decided_abs)) &
        (df["error"].fillna("") == "")
    )

    # Sort worst-first for quick diagnosis
    df = df.sort_values(
        by=["eligible_2025_test", "pct_decided_2025", "has_decision_date_2025", "total_valid_date_2025"],
        ascending=[True, True, True, True],
        na_position="last"
    ).reset_index(drop=True)

    return df

# ---- RUN IT
df_2025 = run_boroughwide_2025_check(min_pct_decided=0.70, min_decided_abs=200)

print(df_2025[[
    "borough", "total_valid_date_2025", "has_decision_date_2025", "missing_decision_date_2025",
    "pct_decided_2025", "top_missing_status", "top_missing_status_pct_of_missing", "eligible_2025_test", "error"
]])

# Save to CSV so you can share or attach later
out_path = "borough_2025_outcome_completeness.csv"
df_2025.to_csv(out_path, index=False)
print("\nSaved:", out_path)


                 borough  total_valid_date_2025  has_decision_date_2025  \
0     Barking & Dagenham                   1029                     188   
1         Waltham Forest                   1798                     517   
2                Hackney                   1534                     599   
3                 Harrow                   1401                     552   
4   Hammersmith & Fulham                   3121                    1713   
5                 Ealing                   2588                    1561   
6         City of London                   1543                     969   
7                 Camden                   1675                    1061   
8               Hounslow                   2773                    1794   
9              Islington                   3140                    2107   
10               Bromley                   2065                    1403   
11              Richmond                   3920                    2881   
12         Tower Hamlets 

In [3]:
import requests

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

def check_status_field(borough, status_field):
    q = {
        "size": 0,
        "track_total_hits": True,
        "query": {"bool": {"must": [
            {"term": {"lpa_name.raw": borough}},
            {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}},
            {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
        ]}},
        "aggs": {"status_breakdown": {"terms": {"field": status_field, "size": 20}}}
    }
    r = requests.post(API, headers=HDR, json=q, timeout=60)
    print("Field:", status_field, "| HTTP:", r.status_code)
    if r.status_code != 200:
        print(r.text)
        return
    buckets = r.json()["aggregations"]["status_breakdown"]["buckets"]
    print("Buckets:", [(b["key"], b["doc_count"]) for b in buckets[:10]])

for f in ["status.raw", "status.keyword", "status"]:
    check_status_field("Barking & Dagenham", f)


Field: status.raw | HTTP: 200
Buckets: [('Application Under Consideration', 841)]
Field: status.keyword | HTTP: 200
Buckets: []
Field: status | HTTP: 400
{"error":{"root_cause":[{"type":"illegal_argument_exception","reason":"Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [status] in order to load field data by uninverting the inverted index. Note that this can use significant memory."}],"type":"search_phase_execution_exception","reason":"all shards failed","phase":"query","grouped":true,"failed_shards":[{"shard":0,"index":"applications","node":"YjOvxhioRamyfuNMwJ22Jg","reason":{"type":"illegal_argument_exception","reason":"Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field 

In [9]:
import requests
import pandas as pd

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

BOROUGHS = [
    "Barking & Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "City of London", "Croydon", "Ealing", "Enfield",
    "Greenwich", "Hackney", "Hammersmith & Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington & Chelsea", "Kingston", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth",
    "Westminster"
]

def borough_2025_completeness(session: requests.Session, borough: str) -> dict:
    """
    Cohort = valid_date in 2025.
    Measures: how many already have decision_date + most common missing status (status.raw).
    """
    try:
        # Total in 2025 by valid_date
        q_total = {
            "size": 0,
            "track_total_hits": True,
            "query": {"bool": {"must": [
                {"term": {"lpa_name.raw": borough}},
                {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}}
            ]}}
        }

        # Those with decision_date present
        q_has_dec = {
            "size": 0,
            "track_total_hits": True,
            "query": {"bool": {"must": [
                {"term": {"lpa_name.raw": borough}},
                {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}},
                {"exists": {"field": "decision_date"}}
            ]}}
        }

        # Missing decision_date -> status breakdown (IMPORTANT: status.raw)
        q_missing_status = {
            "size": 0,
            "track_total_hits": True,
            "query": {"bool": {"must": [
                {"term": {"lpa_name.raw": borough}},
                {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}},
                {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
            ]}},
            "aggs": {
                "status_breakdown": {"terms": {"field": "status.raw", "size": 20, "missing": "__MISSING_STATUS__"}}
            }
        }

        r1 = session.post(API, headers=HDR, json=q_total, timeout=60); r1.raise_for_status()
        r2 = session.post(API, headers=HDR, json=q_has_dec, timeout=60); r2.raise_for_status()
        r3 = session.post(API, headers=HDR, json=q_missing_status, timeout=60); r3.raise_for_status()

        total = int(r1.json()["hits"]["total"]["value"])
        has_dec = int(r2.json()["hits"]["total"]["value"])
        missing = total - has_dec
        pct_decided = (has_dec / total) if total else 0.0

        buckets = r3.json()["aggregations"]["status_breakdown"]["buckets"]
        if missing > 0 and len(buckets) > 0:
            top_status = buckets[0]["key"]
            top_status_pct_of_missing = buckets[0]["doc_count"] / missing
        else:
            top_status = ""
            top_status_pct_of_missing = 0.0

        return {
            "borough": borough,
            "total_valid_date_2025": total,
            "has_decision_date_2025": has_dec,
            "missing_decision_date_2025": missing,
            "pct_decided_2025": pct_decided,
            "top_missing_status": top_status,
            "top_missing_status_pct_of_missing": top_status_pct_of_missing,
            "error": ""
        }

    except Exception as e:
        return {
            "borough": borough,
            "total_valid_date_2025": None,
            "has_decision_date_2025": None,
            "missing_decision_date_2025": None,
            "pct_decided_2025": None,
            "top_missing_status": "",
            "top_missing_status_pct_of_missing": None,
            "error": str(e)
        }

def run_borough_table_2025(out_csv_path="borough_2025_outcome_completeness_FIXED.csv"):
    rows = []
    with requests.Session() as session:
        for b in BOROUGHS:
            rows.append(borough_2025_completeness(session, b))

    df = pd.DataFrame(rows)

    # Example eligibility rule (edit as you like)
    # Here: at least 70% already decided in the 2025 valid_date cohort
    df["eligible_2025_test"] = df["pct_decided_2025"].fillna(0) >= 0.70

    df = df.sort_values(["pct_decided_2025", "total_valid_date_2025"], ascending=[True, False])
    df.to_csv(out_csv_path, index=False)

    print("Saved:", out_csv_path)
    return df

df_fixed = run_borough_table_2025()
print(df_fixed.head(15))


Saved: borough_2025_outcome_completeness_FIXED.csv
                 borough  total_valid_date_2025  has_decision_date_2025  \
0     Barking & Dagenham                   1029                     188   
30        Waltham Forest                   1798                     517   
11               Hackney                   1534                     599   
14                Harrow                   1401                     552   
12  Hammersmith & Fulham                   3133                    1717   
8                 Ealing                   2588                    1561   
6         City of London                   1545                     972   
5                 Camden                   1675                    1061   
17              Hounslow                   2775                    1808   
18             Islington                   3144                    2126   
4                Bromley                   2065                    1403   
26              Richmond                   3920  

In [5]:
import requests
import pandas as pd

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

def missing_status_breakdown(borough: str, size=20):
    q = {
        "size": 0,
        "query": {"bool": {"must": [
            {"term": {"lpa_name.raw": borough}},
            {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}},
            {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
        ]}},
        "aggs": {
            "status_breakdown": {"terms": {"field": "status.raw", "size": size}}
        }
    }
    r = requests.post(API, headers=HDR, json=q, timeout=60)
    r.raise_for_status()
    buckets = r.json()["aggregations"]["status_breakdown"]["buckets"]
    return pd.DataFrame(buckets).rename(columns={"key": "status", "doc_count": "count"})

df_ealing = missing_status_breakdown("Ealing", size=30)
print(df_ealing.head(30))
print("Total missing (sum buckets):", df_ealing["count"].sum())


                            status  count
0  Application Under Consideration     63
Total missing (sum buckets): 63


In [6]:
import requests

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

def missing_decision_status_exists(borough: str):
    base_must = [
        {"term": {"lpa_name.raw": borough}},
        {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}},
        {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
    ]

    q = {
        "size": 0,
        "query": {"bool": {"must": base_must}},
        "aggs": {
            "status_exists": {"filter": {"exists": {"field": "status.raw"}}},
            "status_missing": {"filter": {"bool": {"must_not": [{"exists": {"field": "status.raw"}}]}}},
        }
    }

    r = requests.post(API, headers=HDR, json=q, timeout=60)
    r.raise_for_status()
    aggs = r.json()["aggregations"]

    total_missing_decision = r.json()["hits"]["total"]["value"]
    with_status = aggs["status_exists"]["doc_count"]
    without_status = aggs["status_missing"]["doc_count"]

    return total_missing_decision, with_status, without_status

total, with_status, without_status = missing_decision_status_exists("Ealing")
print("Total missing decision_date:", total)
print("…with status.raw:", with_status)
print("…without status.raw:", without_status)


Total missing decision_date: 1027
…with status.raw: 63
…without status.raw: 964


In [7]:
import requests
import pandas as pd

def missing_status_breakdown_with_missing_bucket(borough: str, size=20):
    q = {
        "size": 0,
        "query": {"bool": {"must": [
            {"term": {"lpa_name.raw": borough}},
            {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}},
            {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}}
        ]}},
        "aggs": {
            "status_breakdown": {
                "terms": {
                    "field": "status.raw",
                    "size": size,
                    "missing": "__MISSING_STATUS__"
                }
            }
        }
    }
    r = requests.post(API, headers=HDR, json=q, timeout=60)
    r.raise_for_status()

    buckets = r.json()["aggregations"]["status_breakdown"]["buckets"]
    df = pd.DataFrame(buckets).rename(columns={"key": "status", "doc_count": "count"})
    return df

df = missing_status_breakdown_with_missing_bucket("Ealing", size=30)
print(df.head(30))
print("Total missing decision_date (sum buckets):", df["count"].sum())


                            status  count
0               __MISSING_STATUS__    964
1  Application Under Consideration     63
Total missing decision_date (sum buckets): 1027


In [8]:
import requests
import pandas as pd

def sample_missing_status_docs(borough: str, n=10):
    q = {
        "size": n,
        "_source": ["id", "lpa_name", "valid_date", "decision_date", "status", "decision"],
        "query": {"bool": {"must": [
            {"term": {"lpa_name.raw": borough}},
            {"range": {"valid_date": {"gte": "01/01/2025", "lt": "01/01/2026"}}},
            {"bool": {"must_not": [{"exists": {"field": "decision_date"}}]}},
            {"bool": {"must_not": [{"exists": {"field": "status.raw"}}]}}
        ]}}
    }
    r = requests.post(API, headers=HDR, json=q, timeout=60)
    r.raise_for_status()
    hits = r.json()["hits"]["hits"]
    return pd.json_normalize([h["_source"] for h in hits])

print(sample_missing_status_docs("Ealing", n=10))


  lpa_name decision_date  valid_date decision                id status
0   Ealing          None  03/06/2025     None  Ealing-252129CND   None
1   Ealing          None  15/05/2025     None  Ealing-251902ADV   None
2   Ealing          None  09/06/2025     None  Ealing-252215CND   None
3   Ealing          None  23/05/2025     None  Ealing-252003CND   None
4   Ealing          None  30/04/2025     None  Ealing-251703ADV   None
5   Ealing          None  04/02/2025     None  Ealing-250457FUL   None
6   Ealing          None  03/06/2025     None  Ealing-252321CND   None
7   Ealing          None  27/02/2025     None  Ealing-250715CPE   None
8   Ealing          None  02/06/2025     None  Ealing-251847FUL   None
9   Ealing          None  15/05/2025     None  Ealing-251513FUL   None
